In [1]:
#  ___________________________________________________________________________
#
#  Pyomo: Python Optimization Modeling Objects
#  Copyright (c) 2008-2025
#  National Technology and Engineering Solutions of Sandia, LLC
#  Under the terms of Contract DE-NA0003525 with National Technology and
#  Engineering Solutions of Sandia, LLC, the U.S. Government retains certain
#  rights in this software.
#  This software is distributed under the 3-clause BSD License.
#  ___________________________________________________________________________
"""
Prototype for multi-experiment optimization using the Rooney-Biegler example.
Testing trace (A-optimality) objective.
"""

import pyomo.environ as pyo
from pyomo.contrib.parmest.experiment import Experiment
from pyomo.contrib.doe import DesignOfExperiments
from pyomo.common.dependencies import pandas as pd, numpy as np


def rooney_biegler_model(data, theta=None):
    model = pyo.ConcreteModel()

    if theta is None:
        theta = {'asymptote': 15, 'rate_constant': 0.5}

    model.asymptote = pyo.Var(initialize=theta['asymptote'])
    model.rate_constant = pyo.Var(initialize=theta['rate_constant'])

    # Fix the unknown parameters
    model.asymptote.fix()
    model.rate_constant.fix()

    # Add the experiment inputs
    model.hour = pyo.Var(initialize=data["hour"].iloc[0], bounds=(0, 10))

    # Fix the experiment inputs
    model.hour.fix()

    # Add the response variable
    model.y = pyo.Var(within=pyo.PositiveReals, initialize=data["y"].iloc[0])

    def response_rule(m):
        return m.y == m.asymptote * (1 - pyo.exp(-m.rate_constant * m.hour))

    model.response_function = pyo.Constraint(rule=response_rule)

    return model


class RooneyBieglerExperiment(Experiment):

    def __init__(self, data, measure_error=None, theta=None):
        self.data = data
        self.model = None
        self.measure_error = measure_error
        self.theta = theta

    def create_model(self):
        # rooney_biegler_model expects a dataframe
        data_df = self.data.to_frame().transpose()
        self.model = rooney_biegler_model(data_df, theta=self.theta)

    def label_model(self):

        m = self.model

        # Add experiment outputs as a suffix
        m.experiment_outputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.experiment_outputs.update([(m.y, self.data['y'])])

        # Add unknown parameters as a suffix
        m.unknown_parameters = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.unknown_parameters.update(
            (k, pyo.value(k)) for k in [m.asymptote, m.rate_constant]
        )

        # Add measurement error as a suffix
        m.measurement_error = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.measurement_error.update([(m.y, self.measure_error)])

        # Add hour as an experiment input
        m.experiment_inputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.experiment_inputs.update([(m.hour, self.data['hour'])])

        # For multiple experiments, add symmetry breaking constraints
        m.sym_break_cons = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.sym_break_cons[m.hour] = None

    def get_labeled_model(self):
        self.create_model()
        self.label_model()
        return self.model


# Data Setup
data = pd.DataFrame(
    data=[[1, 8.3], [2, 10.3], [3, 19.0], [4, 16.0], [5, 15.6], [7, 19.8]],
    columns=['hour', 'y'],
)
theta = {'asymptote': 15, 'rate_constant': 0.5}
measurement_error = 0.1


print(f"Creating {len(data)} experiments...")
FIM_0 = np.zeros((2, 2))
for i in range(len(data)):
    exp_data = data.loc[i, :]
    exp = RooneyBieglerExperiment(
        data=exp_data, theta=theta, measure_error=measurement_error
    )
    doe_obj = DesignOfExperiments(
        experiment_list=exp,
        objective_option='trace',
        prior_FIM=None,
        tee=False,
        _Cholesky_option=True,
        _only_compute_fim_lower=True,
    )
    FIM_0 += doe_obj.compute_FIM()

FIM_0

Creating 6 experiments...


array([[  368.8651408 ,  3410.37384144],
       [ 3410.37384144, 41928.45242007]])

In [2]:
np.linalg.inv(FIM_0)

array([[ 1.09322634e-02, -8.89207753e-04],
       [-8.89207753e-04,  9.61764775e-05]])

In [3]:
np.linalg.det(FIM_0)

np.float64(3835294.7670846786)

In [4]:
# Create DoE object with trace (A-optimality) objective
print(f"\nRunning optimization with trace (A-optimality) objective...")
doe_obj = DesignOfExperiments(
    experiment_list=exp,
    objective_option='determinant',
    prior_FIM=FIM_0,
    tee=False,
    _Cholesky_option=True,
    _only_compute_fim_lower=True,
)
doe_obj.run_doe()


Running optimization with trace (A-optimality) objective...


In [5]:
doe_obj.results

{'Solver Status': <SolverStatus.ok: 'ok'>,
 'Termination Condition': <TerminationCondition.optimal: 'optimal'>,
 'Termination Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found',
 'FIM': [[np.float64(467.52209104009984), np.float64(3510.7624867586915)],
  [np.float64(3510.7624867586915), np.float64(42030.60315638461)]],
 'Sensitivity Matrix': [[np.float64(0.9932620512243722),
   np.float64(1.010696474276815)]],
 'Experiment Design': [9.999999472662282],
 'Experiment Design Names': ['hour'],
 'Experiment Outputs': [14.913829699133277],
 'Experiment Output Names': ['y'],
 'Unknown Parameters': [15.014999999999999, 0.5],
 'Unknown Parameter Names': ['asymptote', 'rate_constant'],
 'Measurement Error': [14.913829699133277],
 'Measurement Error Names': ['y'],
 'Prior FIM': [[np.float64(368.8651407998551), np.float64(3410.3738414381482)],
  [np.float64(3410.3738414381482), np.float64(41928.45242007305)]],
 'Objective expression': 'determinant',
 'log10 A-opt': np.float64(-2.2364249456973684

In [8]:
# Create DoE object with trace (A-optimality) objective
doe_obj = DesignOfExperiments(
    experiment_list=exp,
    objective_option='determinant',
    prior_FIM=FIM_0,
    tee=False,
    _Cholesky_option=True,
    _only_compute_fim_lower=True,
)

# Run optimization
doe_obj.optimize_experiments()

In [7]:
doe_obj.results

{'Solver Status': <SolverStatus.ok: 'ok'>,
 'Termination Condition': <TerminationCondition.optimal: 'optimal'>,
 'Termination Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found',
 'Scenarios': [{'Total FIM': [[467.5220910400933, 3510.7624867586514],
    [3510.7624867586514, 42030.60315638454]],
   'log10 A-opt': np.float64(-2.2364249456973675),
   'log10 pseudo A-opt': np.float64(4.628369772105446),
   'log10 D-opt': np.float64(6.8647947178028135),
   'log10 E-opt': np.float64(2.2381970830910793),
   'FIM Condition Number': np.float64(244.56851784979847),
   'Unknown Parameters': [15.014999999999999, 0.5],
   'Experiments': [{'Experiment Design': [9.999999472662282],
     'Experiment Outputs': [14.913829699133277],
     'Measurement Error': [14.913829699133277],
     'FIM': [[np.float64(98.65695024023822), np.float64(100.38864532050314)],
      [np.float64(100.38864532050314), np.float64(102.150736311484)]],
     'Sensitivity Matrix': [[np.float64(0.9932620512243394),
       np.float6